- n of unique songs
- first song and first discovered artist
- minutes
- n1 artist w/amount of hours
- top artists, top songs, top genres
- top 100
- subgenre
- oldest song

In [1]:
import pandas as pd
import pylast
import time
import datetime
import re
from IPython.display import clear_output


In [2]:
from config import LASTFM_API_KEY, LASTFM_SHARED_SECRET, LASTFM_PASSWORD

In [3]:
# In order to perform a write operation you need to authenticate yourself
username = "alexing10"
password_hash = pylast.md5(LASTFM_PASSWORD)

network = pylast.LastFMNetwork(api_key=LASTFM_API_KEY, api_secret=LASTFM_SHARED_SECRET,
                               username=username, password_hash=password_hash)
user = network.get_authenticated_user()

In [4]:
d = datetime.date(2018,1,1)

unixtime = time.mktime(d.timetuple())
tracks = user.get_recent_tracks(limit=None, time_from=unixtime)

In [5]:
first_elem = tracks[0]
print(type(first_elem), len(tracks))

<class 'pylast.PlayedTrack'> 14812


In [6]:
def print_track(a_track):
    print(".artist: ", a_track.track.artist)
    album = str(a_track.track.get_album())
    if not album:
        album = ''
    print(".get_album(): ", re.sub(r"^[^-]*- ", "", album))
    print(".get_duration(): ",a_track.track.get_duration())
    print(".get_listener_count(): ",a_track.track.get_listener_count())
    print(".get_mbid(): ",a_track.track.get_mbid())
    print(".get_playcount(): ",a_track.track.get_playcount())
    print(".get_url(): ",a_track.track.get_url())
    print(".info: ",a_track.track.info)
    print(".title: ",a_track.track.title)

print_track(first_elem)

.artist:  Marilina Bertoldi
.get_album():  Prender un Fuego
.get_duration():  188000
.get_listener_count():  227
.get_mbid():  None
.get_playcount():  1351
.get_url():  https://www.last.fm/music/marilina%2bbertoldi/_/nunca
.info:  {}
.title:  Nunca


In [7]:
print_track(tracks[568])

.artist:  Riff
.get_album():  None
.get_duration():  0
.get_listener_count():  31
.get_mbid():  None
.get_playcount():  46
.get_url():  https://www.last.fm/music/riff/_/sube%2ba%2bmi%2bvoiture%2b-%2ben%2bvivo
.info:  {}
.title:  Sube a Mi Voiture - En Vivo


In [17]:
def get_data(a_track):
    data = []
    data.append(str(a_track.track.artist))
    data.append(a_track.track.title)
    try:
        album = str(a_track.track.get_album())
        if not album:
            album = ''
    except:
        album = ''
    data.append(re.sub(r"^[^-]*- ", "", album))
    try:
        duration = a_track.track.get_duration()
    except:
        duration = ''
    data.append(duration)
    try:
        listener_count = a_track.track.get_listener_count()
    except:
        listener_count = ''
    data.append(listener_count)
    try:
        mbid = a_track.track.get_mbid()
    except:
        mbid = ''
    data.append(mbid)
    try:
        playcount = a_track.track.get_playcount()
    except:
        playcount = ''
    data.append(playcount)
    try:
        url = a_track.track.get_url()
    except:
        url = ''
    data.append(url)
    data.append(a_track.track.info)
    data.append(a_track.playback_date)
    data.append(a_track.timestamp)
    return data

In [18]:
data = {}
amount_of_tracks = len(tracks)
for i, a_track in enumerate(tracks):
    data[i] = get_data(a_track)
    print("Got data from %d/%d tracks" % (i+1, amount_of_tracks))
    clear_output(wait=True)

Got data from 14812/14812 tracks


In [19]:
df = pd.DataFrame.from_dict(data, orient='index')
df.columns=['artist', 'title', 'album', 'duration', 'listener_count', 'mbid', 'playcount', 'url', 'info', 'date', 'timestamp']
df.head()

,artist,title,album,duration,listener_count,mbid,playcount,url,info,date,timestamp
0,Marilina Bertoldi,Nunca,Prender un Fuego,188000,228,None,1366,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:43",1544298210
1,Marilina Bertoldi,La Casa de A,None,0,243,None,1688,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:40",1544298014
2,Marilina Bertoldi,Fumar de día,Prender un Fuego,178000,350,None,2149,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:37",1544297834
3,Marilina Bertoldi,O No?,Prender un Fuego,181000,242,None,1343,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:34",1544297653
4,The 1975,Inside Your Mind,ネット上の人間関係についての簡単な調査 [Explicit],230000,20179,None,63956,https://www.last.fm/music/the%2b1975/_/inside%...,{},"08 Dec 2018, 18:59",1544295551


In [20]:
df.to_csv("data/data.csv")

In [16]:
a_track.timestamp

'1514804801'

In [13]:
tracks[0]

PlayedTrack(track=pylast.Track('Marilina Bertoldi', 'Nunca', pylast.LastFMNetwork('3217e9e74b50899da477a2e708fbc0af', '7db53e77e3ffe3d7d3026a7dc9d3bf5c', 'F5F0EdDg_-w-IhJclnFrw4DfKuYYXB5q', 'alexing10', '17d2814699b2796ff68bb1b8a4c43edd')), album='Prender un Fuego', playback_date='08 Dec 2018, 19:43', timestamp='1544298210')